In [1]:
#Brain_Segmentation
import os, sys, glob
import SimpleITK as sitk
import sys
import os
import pandas as pd
import re
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from sklearn.utils import shuffle
%matplotlib inline

from skimage.util.dtype import dtype_range
from skimage.util import img_as_ubyte
from skimage import exposure
from skimage.morphology import disk
from skimage.filters import rank
from scipy.ndimage import gaussian_filter
from skimage import data
from skimage import img_as_float
from skimage.morphology import reconstruction
from scipy import ndimage
import random
from random import randrange
from keras.utils import np_utils

from importlib import reload  # Python 3.4+ only.
import visualization_utils  as vu
from visualization_utils import show_lable_on_image4
reload(vu)

import model_unet
reload(model_unet)

IMG_HEIGHT = 240
IMG_WIDTH = 240
IMG_CHANNELS = 4

label_type_shrt = ['background', 'necrotic',
             'edema', 'enhancing']
label_type = ['background', 'necrotic and non-enhancing tumor', 'edema', 'enhancing tumor']

DATA = 'C:/Users/student/Desktop/Naomi_dataset/MICCAI_BraTS_2019_Data_Training/'
VALIDATION_DATA = 'C:/Users/student/Desktop/Naomi_dataset/MICCAI_BraTS_2019_Data_Validation/'
DATA_HGG = DATA +'/HGG/'
DATA_LGG = DATA +'/LGG/'
NUMPY_DIR = 'C:/Users/student/Desktop/Naomi_dataset/train_numpy_images/'
VALIDATION_NUMPY_DIR  = 'C:/Users/student/Desktop/Naomi_dataset/val_numpy_images/'

'''
DATA= 'dicom/MICCAI_BraTS_2019_Data_Training/'
VALIDATION_DATA = 'dicom/MICCAI_BraTS_2019_Data_Validation/'
DATA_HGG = DATA +'/HGG/'
DATA_LGG = DATA +'/LGG/'

NUMPY_DIR = 'dicom/MICCAI_BraTS_2019_Data_Training/numpy_images/'
VALIDATION_NUMPY_DIR = 'dicom/MICCAI_BraTS_2019_Data_Validation/numpy_images/'
'''
FLAIR = 'flair'
T1 = 't1'
T2 = 't2'
T1CE = 't1ce'

img_type=['FLAIR', 'T1','T1CE', 'T2']

model = model_unet.unet(IMG_HEIGHT,IMG_WIDTH,IMG_CHANNELS)
model.load_weights('model_unet_4ch.hdf5')

df_train= pd.read_csv('df_train.csv')
df_test= pd.read_csv('df_test.csv')
df_val= pd.read_csv('df_val.csv')

TEST_PRED_NUMPY_DIR = 'C:/Users/student/Desktop/Naomi_dataset/predictions/pred1/test/numpy_images/'
VALIDATION_PRED_NUMPY_DIR = 'C:/Users/student/Desktop/Naomi_dataset/predictions/pred1/validation/numpy_images/'
VALIDATION_PRED_NII_DIR = 'C:/Users/student/Desktop/Naomi_dataset/predictions/pred1/validation/nii/'



def show_pred_im(im, pred):
    
    vu.show_n_images([im[:,:,1], 
                   im[:,:,0],pred,
                  show_lable_on_image4(im[:,:,1], pred[:,:])],
                 titles=['Flair','T1', 'Pred',  'Prediction on Flair'])

def get_numpy_img_lbl(img_id = 'BraTS19_TCIA10_449_1', np_dir=NUMPY_DIR):
    img=np.load(os.path.join(np_dir, img_id+'.npy'))
    lbl=np.load(os.path.join(np_dir, img_id+'_lbl.npy'))
    return img,lbl

def get_img_for_label(lab=2, axis=0, df=df_train,np_dir = NUMPY_DIR):
    
    img_id= random.choice(df[df['lab'+str(lab)] > 0].id.values)
    
    img,lbl = get_numpy_img_lbl(img_id, np_dir)
    ind = np.where(lbl==lab)
    k = random.randrange(len(ind[0]))
    
    if (axis==0):        
        return img[ind[0][k],:,:] , lbl[ind[0][k],:,:]
        
    lb = np.zeros((240,240),dtype=np.int)
    im = np.zeros((240,240,4),dtype=np.float32)
    
    if (axis==1):
        im[40:40+155,:,:]=img[:, ind[1][k],:,:]
        lb[40:40+155,:]=lbl[:, ind[1][k],:]
        return im,lb
    
    if (axis == 2):
        im[40:40+155,:,:]=img[:, :, ind[2][k],:]
        lb[40:40+155,:]=lbl[:,:,ind[2][k]]
        return im,lb
    return None

def lbl_from_cat(cat_lbl):
    
    lbl=0
    if (len(cat_lbl.shape)==3):
        for i in range(1,4):
            lbl = lbl + cat_lbl[:,:,i]*i
    elif (len(cat_lbl.shape)==4):
        for i in range(1,4):
            lbl = lbl + cat_lbl[:,:,:,i]*i
    else:
        print('Error in lbl_from_cat', cat_lbl.shape)
        return None
    return lbl

def normalize_3D_image(img):
    for z in range(img.shape[0]):
        for k in range(4):
            if (img[z,:,:,k].max()>0):
                img[z,:,:,k] /= img[z,:,:,k].max()
    return img

def get_pred(img, threshold=0.5):
    out_img=img.copy()
    out_img=np.where(out_img>threshold, 1,0)
    return out_img

def prediction_from_probabily_3D(img):
    
    int_image = get_pred(img)
    return lbl_from_cat(int_image)

def predict_3D_img_prob(np_file):
    
    np_img = np.load(np_file)
    for_pred_img = np.zeros((155, 240, 240, 4), np.float32)

    # Normalize image
    for_pred_img = normalize_3D_image(np_img)

    mdl_pred_img =  model.predict(for_pred_img)

    #pred_label = prediction_from_probabily_3D(mdl_pred_img)

    return mdl_pred_img




Using TensorFlow backend.


In [2]:
#img_id='BraTS19_2013_1_1'
#nimg = os.path.join(VALIDATION_NUMPY_DIR, img_id+'.npy')
#im = np.load(nimg)
#pred_stats = predict_3D_img_prob(nimg)
#pred = prediction_from_probabily_3D(pred_stats)
#show_pred_im(im[88], pred[88])

#Pyqt
from PyQt5.QtWidgets import *
from PyQt5 import uic
from PyQt5 import QtCore, QtGui, QtWidgets 
import cv2
import time
import numpy as np
import datetime
import sys
from os import path
from flask import Flask
from collections import deque
from flask import Flask
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
#run_with_ngrok(app)

@app.route("/", methods=['POST', 'GET'])

#웹페이지 템플릿
def home(): 
    return '''
            <html lang="en">
            <head>
                <meta charset="UTF-8">
                <title>Brain_Tumor_Segmentation</title>

            </head>
            <body>
            <style>
            H1 {
              position: absolute;
              left: 245px;
              top: 100px;
              color:black;
            }
            H2 {
              position: absolute;
              left: 250px;
              top: 200px;
              color:black;
            }
            H3 {
              position: absolute;
              left: 420px;
              top: 650px;
              color:black;
            }
            </style>
            <H1>Brain Tumor Segmentation</H1>
            <H2>
            <p>
            <img src="graph.PNG" width="400" height="400">
            </p>
            </H2>

            <H3><form action="/BTS/" method="post"><button name="forwardBtn" type="submit">BTS.exe</button></H3></form>
            </body>
            </html>
            '''

@app.route("/BTS/", methods=['POST', 'GET'])

#메인함수
def BTS():
    #UI파일 연결
    #단, UI파일은 Python 코드 파일과 같은 디렉토리에 위치해야한다.
    form_class = uic.loadUiType("BTS_controller.ui")[0]
    ul = False
    q = False

    #화면을 띄우는데 사용되는 Class 선언
    class WindowClass(QMainWindow, form_class) :

        def __init__(self) :

            super().__init__()
            self.setupUi(self)

            #버튼에 기능을 연결하는 코드
            self.pushButton.clicked.connect(self.MRI_Upload)
            self.pushButton_2.clicked.connect(self.Exit)  
            
        def setupUi(self, Dialog):    
            self.graphicsView(Dialog)
            
        #Brain_Tumor_Segmentation
        def BTS_Service(self):
            global ul
            global q
            
            ul = False
            q = False

            while True:
                if(ul == True and q==False):
                    nimg = MRI_Upload()
                    im = np.load(nimg)
                    pred_stats = predict_3D_img_prob(nimg)
                    pred = prediction_from_probabily_3D(pred_stats)
                    show_pred_im(im[88], pred[88])

                if ((q == True) or (key == 113)): #q
                    break

                if not ret:
                    break

                cv2.destroyAllWindows()
            
        #파일 업로드
        def MRI_Upload(self):
            global ul
            ul = True
            fname = QFileDialog.getOpenFileName(self, 'Open file', '/')
            if fname[0]:

                #f = open(fname[0], 'r', encoding='UTF-8')
                #flines = f.readlines()
                f = open(fname[0], 'r', encoding='UTF-8')
                return f

               # for line in flines:
                #    print(line)
                #nimg = os.path.join(VALIDATION_NUMPY_DIR, img_id+'.npy')
                #im = np.load(f)
                #pred_stats = predict_3D_img_prob(nimg)
                #pred = prediction_from_probabily_3D(pred_stats)
                #show_pred_im(im[88], pred[88])
                
            else:
                QMessageBox.about(self, "Warning", "파일을 선택하지 않았습니다.")
                return f
         
        #이미지 보이기
        '''
        def loadImageFromFile(self) :
            #QPixmap 객체 생성 후 이미지 파일을 이용하여 QPixmap에 사진 데이터 Load하고, Label을 이용하여 화면에 표시
            self.qPixmapFileVar = QPixmap()
            self.qPixmapFileVar.load("pick.png")
            self.qPixmapFileVar = self.qPixmapFileVar.scaledToWidth(600)
            self.lbl_picture.setPixmap(self.qPixmapFileVar)
        '''

                
            
        #프로그램 종료
        def Exit(self):
            global q
            q = True
                

    if __name__ == "__main__" :
        app = QApplication(sys.argv)
        mw = WindowClass() 
        mw.show()
        app.exec_()
        
    return home()
        
if __name__ == "__main__" :
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [12/Jun/2020 17:51:21] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 17:51:21] "GET /graph.PNG HTTP/1.1" 404 -
[2020-06-12 17:51:22,923] ERROR in app: Exception on /BTS/ [POST]
Traceback (most recent call last):
  File "C:\Users\student\.conda\envs\test\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\student\.conda\envs\test\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\student\.conda\envs\test\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\student\.conda\envs\test\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\student\.conda\envs\test\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\